In [ ]:
import numpy as np
import os
import cv2
import random
import json
from tqdm.notebook import tqdm
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
# from detectron2.utils.visualizer import Visualizer
# from detectron2.data.catalog import DatasetCatalog

In [ ]:
register_coco_instances('train', {}, 'data/train.json', 'data/train_images/')
register_coco_instances('val', {}, 'data/val.json', 'data/val_images/')
register_coco_instances('test', {}, 'data/test.json', 'data/test_images/')

In [ ]:
class COCOTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs('coco_eval', exist_ok=True)
            output_folder = 'coco_eval'
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
config = get_cfg()
config.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml'))
config.DATASETS.TRAIN = ('train',)
config.DATASETS.TEST = ('val',)

config.DATALOADER.NUM_WORKERS = 8
config.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml')  # Let training initialize from model zoo
config.SOLVER.IMS_PER_BATCH = 4
config.SOLVER.BASE_LR = 0.001

config.SOLVER.WARMUP_ITERS = 1000
config.SOLVER.MAX_ITER = 2000 #adjust up if val mAP is still rising, adjust down if overfit
config.SOLVER.STEPS = (1000, 1500)
config.SOLVER.GAMMA = 0.05

config.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
config.MODEL.ROI_HEADS.NUM_CLASSES = 2 #your number of classes + 1

config.TEST.EVAL_PERIOD = 500

In [ ]:
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

In [ ]:
trainer = COCOTrainer(config)
trainer.resume_or_load(resume=False)

In [ ]:
trainer.train()

In [ ]:
config.MODEL.WEIGHTS = os.path.join(config.OUTPUT_DIR, 'model_final.pth')
config.DATASETS.TEST = ('test',)
config.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

In [ ]:
predictor = DefaultPredictor(config)

In [ ]:
with open('data/test.json') as json_file:
    data = json.load(json_file)

In [ ]:
test_images_map_id={}
for x in data['images']:
    test_images_map_id[x['file_name']] = x['id']

In [ ]:
files = os.listdir('data/test_images')
_final_object = []
for f in tqdm(files, total=len(files)):
    try:
        images = cv2.imread('data/test_images/' + f)
        predictions = predictor(images)['instances'].get_fields()
        classes = predictions['pred_classes'].tolist()
        scores = predictions['scores'].tolist()
        bbox = predictions['pred_boxes'].tensor.tolist()
        
        for idx in range(len(classes)):
            c = classes[idx]
            s = scores[idx]
            b = bbox[idx]
            _result = {}
            _result['image_id'] = test_images_map_id[f]
            _result['bbox'] = [b[0], b[1], b[2] - b[0], b[3] - b[1]]
            _result['score'] = s
            _result['category_id'] = c + 1
            _final_object.append(_result)

    except:
        print('Error with',f)
        continue

In [ ]:
fp = open('data/submission.json', 'w')
print('Writing JSON...')
fp.write(json.dumps(_final_object))
fp.close()